In [1]:
# Monte-Carlo playoff odds
# Generate my own playoff odds

# For now, I'm focusing on the mechanics of the simulation, and less so on the inputs (e.g., the projected team quality)
# So I'm using 538's win probabilities for each game, rather than computing my own

# I'm also using 538's results/schedule data, because it is so easy to use

import pandas as pd
import numpy as np

In [2]:
# Read in the 538 dataset, which has a row for each game in the current season (played or unplayed)
gms = pd.read_csv('https://projects.fivethirtyeight.com/mlb-api/mlb_elo_latest.csv')
#gms = pd.read_csv('../data/538/mlb-elo/mlb_elo_latest.csv')
gms

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,2022-10-05,2022,0,NaN,LAD,COL,1604.797760,1472.520826,0.710868,0.289132,...,NaN,NaN,NaN,NaN,0.723387,0.276613,NaN,NaN,NaN,NaN
1,2022-10-05,2022,0,NaN,SEA,DET,1524.337226,1451.512688,0.635843,0.364157,...,NaN,NaN,NaN,NaN,0.622726,0.377274,NaN,NaN,NaN,NaN
2,2022-10-05,2022,0,NaN,SDP,SFG,1511.860691,1531.550690,0.506202,0.493798,...,NaN,NaN,NaN,NaN,0.559583,0.440417,NaN,NaN,NaN,NaN
3,2022-10-05,2022,0,NaN,NYM,WSN,1535.209697,1431.603957,0.675805,0.324195,...,NaN,NaN,NaN,NaN,0.683656,0.316344,NaN,NaN,NaN,NaN
4,2022-10-05,2022,0,NaN,MIL,ARI,1520.117053,1466.163589,0.610339,0.389661,...,NaN,NaN,NaN,NaN,0.628387,0.371613,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2022-04-07,2022,0,NaN,ATL,CIN,1555.630840,1501.967218,0.609942,0.390058,...,58.198554,53.297336,18.664382,15.512738,0.620108,0.379892,1552.570297,1501.193092,3.0,6.0
2426,2022-04-07,2022,0,NaN,WSN,NYM,1476.319846,1495.202033,0.507365,0.492635,...,46.506602,48.182760,-10.890192,-33.183129,0.495889,0.504111,1467.302390,1522.210391,1.0,5.0
2427,2022-04-07,2022,0,NaN,STL,PIT,1524.880454,1456.114951,0.630416,0.369584,...,57.273136,46.669517,27.921385,2.182563,0.650312,0.349688,1503.439418,1444.031029,9.0,0.0
2428,2022-04-07,2022,0,NaN,KCR,CLE,1480.923133,1501.256999,0.505276,0.494724,...,50.288294,59.572636,7.862364,30.139987,0.476089,0.523911,1473.144618,1491.474766,3.0,1.0


In [3]:
gms.columns

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'rating1_pre', 'rating2_pre', 'pitcher1', 'pitcher2', 'pitcher1_rgs',
       'pitcher2_rgs', 'pitcher1_adj', 'pitcher2_adj', 'rating_prob1',
       'rating_prob2', 'rating1_post', 'rating2_post', 'score1', 'score2'],
      dtype='object')

In [4]:
# Split out the games that have been played vs those remaining
played = gms.dropna(subset=['score1']) # games that have a score
remain = gms.loc[gms.index.difference(played.index)] # all other games
played.shape, remain.shape

((1435, 26), (995, 26))

# Define some functions that will be used in the simulation

In [5]:
def compute_standings(gms_played):
    margins = gms_played['score1']-gms_played['score2']
    winners = pd.Series(np.where(margins>0, gms_played['team1'], gms_played['team2']))
    losers  = pd.Series(np.where(margins<0, gms_played['team1'], gms_played['team2']))
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    return standings

compute_standings(played)

,W,L
NYY,66,31
HOU,64,32
LAD,64,30
NYM,59,37
ATL,58,39
SDP,54,43
MIL,53,43
TOR,53,43
TBD,52,43
MIN,52,44


In [6]:
# This is the source data for the mapping of teams to divisions/leagues
div_text = '''
NLW: ARI COL LAD SDP SFG
NLE: ATL FLA NYM PHI WSN
ALW: SEA ANA HOU OAK TEX
ALE: TBD TOR BAL NYY BOS
ALC: MIN CHW CLE KCR DET
NLC: STL MIL CHC PIT CIN
'''

divs = {line.split(': ')[0]: line.split(': ')[1].split(' ') for line in div_text.strip().split('\n')}
teams = pd.DataFrame(pd.concat([pd.Series({team: div for team in teams}) for (div, teams) in divs.items()]).rename('div'))
teams['lg'] = teams['div'].str[0]
teams

,div,lg
ARI,NLW,N
COL,NLW,N
LAD,NLW,N
SDP,NLW,N
SFG,NLW,N
ATL,NLE,N
FLA,NLE,N
NYM,NLE,N
PHI,NLE,N
WSN,NLE,N


In [7]:

def sim_rem_games(remain):
    # Generate a random number for each game
    randoms = pd.Series(np.random.rand(len(remain)), index=remain.index)

    # Figure out the winners and losers
    winners = pd.Series(np.where(randoms<remain['rating_prob1'], remain['team1'], remain['team2']))
    losers = pd.Series(np.where(randoms>remain['rating_prob1'], remain['team1'], remain['team2']))

    # Compute and return the standings
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    for col in standings.columns: # convert to int
        standings[col] = standings[col].fillna(0).astype(int)
    return standings

sim_rem_games(remain)

,W,L
LAD,46,22
SEA,44,22
HOU,43,23
ATL,40,25
MIL,39,27
BOS,38,28
NYM,38,28
TBD,37,30
NYY,37,28
PHI,37,30


In [8]:
cur_standings = compute_standings(played)
rem_standings = sim_rem_games(remain)
full_standings = cur_standings+rem_standings
full_standings

,W,L
ANA,73,89
ARI,66,96
ATL,94,68
BAL,83,79
BOS,76,86
CHC,66,96
CHW,83,79
CIN,73,89
CLE,83,79
COL,73,89


In [9]:
# find playoff teams
def add_playoff_seeds(standings):
    standings['wpct'] = standings['W'] / (standings['W'] + standings['L'])

    # Merge in the div/lg data
    standings['div'] = teams['div']
    standings['lg'] = teams['lg']

    # Rather than model out all the tie-breakers, I'm assuming that they are all random (not exactly true, but close enough),
    # and so I'm just generating a random number for each team, and we break ties by comparing that random num for each of the tied teams.
    # This is *so* much simpler and faster than modeling all the different scenarios.
    # It might be worth modeling them out with 1-2 days left in the season, but for most of the season, I way prefer using the random num to break ties
    standings['rand'] = np.random.rand(len(standings))

    # Now sort, and break ties using the rand
    sorted = standings.sort_values(by=['wpct', 'rand'], ascending=False)

    # div_rank is nice to have, but somewhat expensive to compute
    #standings['div_rank'] = sorted.groupby('div').cumcount()+1
    #standings['div_win'] = standings['div_rank'] == 1

    # Set div_win False as default, then set it True for div winners
    standings['div_win'] = False
    standings.loc[sorted.groupby('div').head(1).index, 'div_win'] = True
    standings['lg_rank'] = standings.sort_values(by=['div_win', 'wpct', 'rand'], ascending=False).groupby('lg').cumcount()+1
    return standings.sort_values(['lg', 'lg_rank'])

     

add_playoff_seeds(full_standings)

,W,L,wpct,div,lg,rand,div_win,lg_rank
HOU,110,52,0.679012,ALW,A,0.155751,True,1
NYY,100,62,0.617284,ALE,A,0.148280,True,2
MIN,84,78,0.518519,ALC,A,0.883797,True,3
TOR,93,69,0.574074,ALE,A,0.828437,False,4
SEA,89,73,0.549383,ALW,A,0.519068,False,5
CLE,83,79,0.512346,ALC,A,0.983856,False,6
CHW,83,79,0.512346,ALC,A,0.166627,False,7
BAL,83,79,0.512346,ALE,A,0.126428,False,8
TBD,81,81,0.500000,ALE,A,0.450218,False,9
BOS,76,86,0.469136,ALE,A,0.109783,False,10


In [10]:
%%prun -s cumulative # This runs the code profiler, which creates data I can use to find opportunities for me to speed up the code

[add_playoff_seeds(full_standings) for _ in range(1000)]
None # This is to suppress printing the output, which is 1000 lines of the same list of teams

         11604933 function calls (11473933 primitive calls) in 5.866 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.866    5.866 {built-in method builtins.exec}
        1    0.004    0.004    5.866    5.866 <string>:3(<module>)
        1    0.004    0.004    5.862    5.862 <string>:3(<listcomp>)
     1000    0.029    0.000    5.858    0.006 <ipython-input-9-f5dfa596ebf3>:2(add_playoff_seeds)
     3000    0.005    0.000    2.530    0.001 _decorators.py:302(wrapper)
     3000    0.025    0.000    2.522    0.001 frame.py:6269(sort_values)
     3000    0.094    0.000    1.776    0.001 sorting.py:285(lexsort_indexer)
     7000    0.051    0.000    1.296    0.000 categorical.py:365(__init__)
    16000    0.043    0.000    1.252    0.000 frame.py:3463(__getitem__)
     1000    0.005    0.000    1.183    0.001 groupby.py:3040(cumcount)
51000/35000    0.030    0.000    0.892    0.000 groupby.py:90

In [11]:
def finish_one_season(incoming_standings, remain):
    rem_standings = sim_rem_games(remain)
    full_standings = incoming_standings+rem_standings
    full_standings = add_playoff_seeds(full_standings)
    return full_standings

finish_one_season(cur_standings, remain)

,W,L,wpct,div,lg,rand,div_win,lg_rank
HOU,109,53,0.672840,ALW,A,0.606315,True,1
NYY,100,62,0.617284,ALE,A,0.590844,True,2
MIN,89,73,0.549383,ALC,A,0.728491,True,3
TOR,91,71,0.561728,ALE,A,0.672176,False,4
TBD,86,76,0.530864,ALE,A,0.151823,False,5
BOS,85,77,0.524691,ALE,A,0.993683,False,6
CLE,85,77,0.524691,ALC,A,0.312108,False,7
CHW,85,77,0.524691,ALC,A,0.239516,False,8
SEA,84,78,0.518519,ALW,A,0.436982,False,9
BAL,75,87,0.462963,ALE,A,0.288744,False,10


In [12]:

def sim_1_season(incoming_standings, remain, i):
    standings = finish_one_season(incoming_standings, remain)
    standings['iter'] = i
    standings = standings.reset_index().rename(columns={'index': 'team'}).set_index(['team', 'iter'])
    return standings

def sim_n_seasons(incoming_standings, remain, n):
    return pd.concat([sim_1_season(incoming_standings, remain, i) for i in range(n)])

sim_results = sim_n_seasons(cur_standings, remain, 10)
sim_results

,,W,L,wpct,div,lg,rand,div_win,lg_rank
team,iter,,,,,,,,
HOU,0,107,55,0.660494,ALW,A,0.640564,True,1
NYY,0,102,60,0.629630,ALE,A,0.285363,True,2
MIN,0,87,75,0.537037,ALC,A,0.769988,True,3
TBD,0,90,72,0.555556,ALE,A,0.168951,False,4
TOR,0,89,73,0.549383,ALE,A,0.795296,False,5
...,...,...,...,...,...,...,...,...,...
CHC,9,69,93,0.425926,NLC,N,0.881390,False,11
PIT,9,69,93,0.425926,NLC,N,0.684529,False,12
CIN,9,65,97,0.401235,NLC,N,0.363570,False,13


In [13]:
# Count the number of div/wc/playoff appearances by team from a set of results

# Championship weights by seed position
weights = {i: 1/16 for i in range(1,7)}
weights[1] = 1/8
weights[2] = 1/8

def summarize_sim_results(df_results):
    counts = df_results.query('lg_rank <= 6').reset_index()[['team', 'lg_rank']].value_counts().unstack()
    mean_wins = sim_results.groupby('team')['W'].mean().rename('mean_wins')
    summary = pd.merge(left=mean_wins, right=counts, on='team', how='left')
    for col in counts.columns:
        summary[col] = summary[col].fillna(0).astype(int)    

    summary['div_wins'] = summary[range(1, 4)].sum(axis=1)
    summary['playoffs'] = summary[range(1, 7)].sum(axis=1)
    summary['champ_shares'] = (summary[range(1,7)] * np.array(weights)).sum(axis=1)
    return summary

summarize_sim_results(sim_results)

,mean_wins,1,2,3,4,5,6,div_wins,playoffs,champ_shares
team,,,,,,,,,,
ANA,72.4,0,0,0,0,0,0,0,0,0.0000
ARI,68.0,0,0,0,0,0,0,0,0,0.0000
ATL,95.6,0,2,1,4,3,0,3,10,0.7500
BAL,77.2,0,0,0,0,0,0,0,0,0.0000
BOS,81.5,0,0,0,1,0,0,0,1,0.0625
CHC,67.9,0,0,0,0,0,0,0,0,0.0000
CHW,87.0,0,0,4,0,0,1,4,5,0.3125
CIN,67.7,0,0,0,0,0,0,0,0,0.0000
CLE,81.0,0,0,0,0,0,2,0,2,0.1250


In [14]:
#%%prun -s cumulative # This runs the code profiler, which creates data I can use to find opportunities for me to speed up the code

sim_results = sim_n_seasons(cur_standings, remain, 10*1000)
summarize_sim_results(sim_results)

,mean_wins,1,2,3,4,5,6,div_wins,playoffs,champ_shares
team,,,,,,,,,,
ANA,73.1783,0,0,0,0,10,27,0,37,2.3125
ARI,69.9781,0,0,0,0,0,3,0,3,0.1875
ATL,95.3361,140,3883,182,4269,1140,294,4205,9908,870.6875
BAL,75.9739,0,0,0,11,43,141,0,195,12.1875
BOS,81.4086,0,0,0,197,649,1130,0,1976,123.5000
CHC,69.1647,0,0,1,0,0,0,1,1,0.0625
CHW,85.4136,0,8,4048,134,639,1151,4056,5980,374.2500
CIN,68.3898,0,0,0,0,0,0,0,0,0.0000
CLE,82.0930,0,1,1397,61,410,908,1398,2777,173.6250


In [15]:
sim_results.groupby('iter')['W'].max().median()

109.0

In [16]:
summary = summarize_sim_results(sim_results)
print(summary.to_string())

      mean_wins     1     2     3     4     5     6  div_wins  playoffs  champ_shares
team                                                                                 
ANA     73.1783     0     0     0     0    10    27         0        37        2.3125
ARI     69.9781     0     0     0     0     0     3         0         3        0.1875
ATL     95.3361   140  3883   182  4269  1140   294      4205      9908      870.6875
BAL     75.9739     0     0     0    11    43   141         0       195       12.1875
BOS     81.4086     0     0     0   197   649  1130         0      1976      123.5000
CHC     69.1647     0     0     1     0     0     0         1         1        0.0625
CHW     85.4136     0     8  4048   134   639  1151      4056      5980      374.2500
CIN     68.3898     0     0     0     0     0     0         0         0        0.0000
CLE     82.0930     0     1  1397    61   410   908      1398      2777      173.6250
COL     69.8684     0     0     0     0     0     1   

In [17]:
# How many games does each team win in each seeding?
sim_results.query('lg_rank <= 6').groupby(['team', 'lg_rank'])['W'].mean().unstack()

lg_rank,1,2,3,4,5,6
team,,,,,,
ANA,NaN,NaN,NaN,NaN,85.300000,84.259259
ARI,NaN,NaN,NaN,NaN,NaN,82.000000
ATL,102.721429,98.212207,94.318681,94.338955,91.138596,88.221088
BAL,NaN,NaN,NaN,88.545455,86.162791,84.560284
BOS,NaN,NaN,NaN,89.837563,87.342065,85.436283
CHC,NaN,NaN,83.000000,NaN,NaN,NaN
CHW,NaN,94.875000,88.425395,88.977612,87.190923,85.485665
CLE,NaN,94.000000,87.476020,88.245902,86.814634,85.126652
COL,NaN,NaN,NaN,NaN,NaN,85.000000


In [18]:
# How many wins do teams have in division-winning seasons?
sim_results.query('div_win').groupby('team')['W'].mean()

team
ATL     98.193817
CHC     83.000000
CHW     88.438116
CLE     87.480687
HOU    104.266540
LAD    107.803464
MIL     90.589610
MIN     88.766168
NYM     98.624277
NYY    105.466399
PHI     93.941860
SDP     97.000000
SEA     96.555556
STL     89.297652
TBD     97.500000
TOR     98.487805
Name: W, dtype: float64

In [19]:
# How often do teams win the division when they win 95 games?
finishes = sim_results.query('W>=95').groupby('team').agg(num_seasons=('div_win', len), div_wins=('div_win', sum))
finishes['pct_win'] = finishes['div_wins']/finishes['num_seasons']
finishes


,num_seasons,div_wins,pct_win
team,,,
ATL,5816,3732,0.641678
BOS,6,0,0.000000
CHW,95,95,1.000000
CLE,12,12,1.000000
HOU,9935,9928,0.999295
LAD,9995,9986,0.999100
MIL,903,901,0.997785
MIN,163,163,1.000000
NYM,6943,5186,0.746939


In [20]:
pads95 = sim_results.query('team=="SDP" and W>=95').reset_index()['iter']
sim_results.query('iter in @pads95 and div=="NLW"').groupby('team')['W'].mean()

team
ARI     69.361522
COL     68.961945
LAD    106.445032
SDP     96.361522
SFG     81.639535
Name: W, dtype: float64